In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils.np_utils import to_categorical

In [41]:
max_words = 15
maxlen = 10
texts = ['お 時間 を 頂戴 し て おり ます', '時間 が あり ま せん']

# word indexを作成
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
print(sequences)

[[2, 1, 3, 4, 5, 6, 7, 8], [1, 9, 10, 11, 12]]


In [38]:
word_index = tokenizer.word_index
print("Found {} unique tokens.".format(len(word_index)))

data = pad_sequences(sequences, maxlen=maxlen)

Found 12 unique tokens.


In [39]:
print(data)
print(data.shape[0])
indices = np.arange(data.shape[0])
print(indices)

[[0 0 2 1 3 4 5 6 7 8]
 [0 0 0 0 0 0 0 0 1 9]]
2
[0 1]


In [10]:
# バイナリの行列に変換
categorical_labels = to_categorical(labels)
labels = np.asarray(categorical_labels)

print("Shape of data tensor:{}".format(data.shape))
print("Shape of label tensor:{}".format(labels.shape))

NameError: name 'labels' is not defined

In [11]:
import os
import re
import io
import requests
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from zipfile import ZipFile

In [12]:
sess = tf.Session()
epochs = 20
batch_size = 250
max_sequence_length = 25
rnn_size = 10
embedding_size = 50
min_word_frequency = 10
learning_rate = 0.0005
dropout_keep_prob = tf.placeholder(tf.float32)

In [16]:
data_dir = 'temp'
data_file = 'text_data.txt'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
if not os.path.isfile(os.path.join(data_dir, data_file)):
        zip_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/' \
                  '00228/smsspamcollection.zip'
        r = requests.get(zip_url)
        z = ZipFile(io.BytesIO(r.content))
        file = z.read('SMSSpamCollection')
        
        # データの書式を設定
        text_data = file.decode()
        text_data = text_data.encode('ascii', errors='ignore') 
        text_data = text_data.decode().split('\n')
        # データをテキストファイルに保存
        with open(os.path.join(data_dir, data_file), 'w') as file_conn:
            for text in text_data:
                file_conn.write("{}\n".format(text))
else:
        # テキストファイルからデータを読み込む
        text_data = []
        with open(os.path.join(data_dir, data_file), 'r') as file_conn:
                    for row in file_conn:
                        text_data.append(row)
        text_data = text_data[:-1]
text_data = [x.split('\t') for x in text_data if len(x)>=1] 
[text_data_target, text_data_train] = [list(x) for x in zip(*text_data)]

In [18]:
def clean_text(text_string):
    text_string = re.sub(r'([^\s\w]|_|[0-9])+', '', text_string) 
    text_string = " ".join(text_string.split())
    text_string = text_string.lower()
    return(text_string)

# テキストをクリーニング
text_data_train = [clean_text(x) for x in text_data_train]

In [19]:
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
        max_sequence_length, min_frequency=min_word_frequency)
text_processed =  np.array(list(vocab_processor.fit_transform(text_data_train)))

In [40]:
print(text_processed[2])

[ 46 465   9   4 773 882   1 178   0   0 622   0 236 258  69   0   1   1
 318 465   0  77   0 368   0]
